In [4]:
import torch

print("torch version:", torch.__version__)
print("torch is available:", torch.cuda.is_available())

torch version: 2.8.0+cu128
torch is available: True


Pixel8M

In [ ]:
import pandas as pd

df_information = pd.read_csv('./information/Pixel8M/Pixel8M.csv')

print("DataFrame Information:")
print("\nFirst 5 rows of the DataFrame:")
print(df_information.head())

In [ ]:
df_interactions = pd.read_csv('./interactions/Pixel8M/Pixel8M.csv')
print("\nFirst 5 rows of the Interactions DataFrame:")
print(df_interactions.head())

MovieLens

用户信息文件（Users.dat）:<br>
UserID：用户唯一标识。从1~6040， 代表了6040个MovieLens用户<br>Gender：性别（M表示男性，F表示女性）<br>Age：用户年龄，分成了7组<br>Occupation：用户职业，如学生、教师、工程师等<br>Zip-code：用户所在地区的邮政编码
<br><br>
电影信息文件（Movies.dat）:<br>
MovieID：电影唯一标识。从1~3952， 代表了3952部电影<br>
Title：电影标题，通常包含电影名称和发行年份。<br>
Genres：电影题材由竖线分开， 包括动作喜剧等18种电影类型，如“Action|Comedy”<br>
<br><br>
评分文件（Ratings.dat）<br>
UserID：用户唯一标识<br>
MovieID：电影唯一标识<br>
Rating：用户对电影的评分，通常为1到5的整数<br>
Timestamp：评分的时间戳，表示自1970年1月1日以来的秒数<br>

In [26]:
import pandas as pd

# 用户基本信息
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
user_df = pd.read_csv('./MovieLens-1m/users.dat',
                      sep='::',
                      header=None,
                      names=unames,
                      engine='python')

# 电影信息
mnames = ['movie_id', 'title', 'genres']
movies_df = pd.read_csv('./MovieLens-1m/movies.dat',
                        sep='::',
                        header=None,
                        names=mnames,
                        engine='python',
                        encoding='ISO-8859-1')

# 评分信息
rnames = ['user_id', 'movie_id', 'imdbId', 'timestamp']
ratings_df = pd.read_csv('./MovieLens-1m/ratings.dat',
                         sep='::',
                         header=None,
                         engine='python',
                         names=rnames)

In [15]:
user_df.head()

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [16]:
movies_df.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
ratings_df.head()

,user_id,movie_id,imdbId,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [27]:
# 处理电影标题，去掉年份
import re
patter = re.compile(r'^(.*)\((\d+)\)$')
title = {val:patter.match(val).group(1) for i,val in enumerate(set(movies_df['title']))}
movies_df['title'] = movies_df['title'].map(title)  
movies_df.head()

,movie_id,title,genres
0,1,Toy Story,Animation|Children's|Comedy
1,2,Jumanji,Adventure|Children's|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama
4,5,Father of the Bride Part II,Comedy


In [28]:
# 将时间戳转换为日期时间格式
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'],unit='s')

# 重命名列
ratings_df.rename(columns={'timestamp':'time'},inplace=True)
ratings_df.head()

,user_id,movie_id,imdbId,time
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


In [29]:
# 提取日期部分
import datetime
date_df = pd.DataFrame({'time':ratings_df['time']})
date_df['date'] = pd.to_datetime(date_df['time'])
ratings_df['date'] = date_df['date'].apply(lambda x:x.strftime('%Y-%m-%d'))
ratings_df.head()

,user_id,movie_id,imdbId,time,date
0,1,1193,5,2000-12-31 22:12:40,2000-12-31
1,1,661,3,2000-12-31 22:35:09,2000-12-31
2,1,914,3,2000-12-31 22:32:48,2000-12-31
3,1,3408,4,2000-12-31 22:04:35,2000-12-31
4,1,2355,5,2001-01-06 23:38:11,2001-01-06


In [30]:
# 子数据合并
movies_ratings_df = pd.merge(ratings_df,movies_df,on='movie_id')
movies_ratings_df.head()

,user_id,movie_id,imdbId,time,date,title,genres
0,1,1193,5,2000-12-31 22:12:40,2000-12-31,One Flew Over the Cuckoo's Nest,Drama
1,1,661,3,2000-12-31 22:35:09,2000-12-31,James and the Giant Peach,Animation|Children's|Musical
2,1,914,3,2000-12-31 22:32:48,2000-12-31,My Fair Lady,Musical|Romance
3,1,3408,4,2000-12-31 22:04:35,2000-12-31,Erin Brockovich,Drama
4,1,2355,5,2001-01-06 23:38:11,2001-01-06,"Bug's Life, A",Animation|Children's|Comedy


In [31]:
movies_ratings_user_df = pd.merge(pd.merge(ratings_df,movies_df),user_df)
movies_ratings_user_df.head()

,user_id,movie_id,imdbId,time,date,title,genres,gender,age,occupation,zip
0,1,1193,5,2000-12-31 22:12:40,2000-12-31,One Flew Over the Cuckoo's Nest,Drama,F,1,10,48067
1,1,661,3,2000-12-31 22:35:09,2000-12-31,James and the Giant Peach,Animation|Children's|Musical,F,1,10,48067
2,1,914,3,2000-12-31 22:32:48,2000-12-31,My Fair Lady,Musical|Romance,F,1,10,48067
3,1,3408,4,2000-12-31 22:04:35,2000-12-31,Erin Brockovich,Drama,F,1,10,48067
4,1,2355,5,2001-01-06 23:38:11,2001-01-06,"Bug's Life, A",Animation|Children's|Comedy,F,1,10,48067
